In [ ]:
from openai import AsyncAzureOpenAI
from agents import (
    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    function_tool,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)
from agents.run import RunConfig
import os
from dotenv import load_dotenv
from pathlib import Path
import logging

In [22]:
# Get the absolute path to the .env file
current_dir = Path().absolute()
env_path = current_dir / ".env"
print(f"Looking for .env file at: {env_path}")
print(f"Does .env file exist? {env_path.exists()}")

# Load environment variables from .env file with explicit path
load_dotenv(dotenv_path=env_path)

# Debug environment variables to ensure they're loaded
print(
    f"API KEY: {os.getenv('AZURE_OPENAI_API_KEY', '')[:5] if os.getenv('AZURE_OPENAI_API_KEY') else 'Not found'}..."
)
print(f"API VERSION: {os.getenv('AZURE_OPENAI_API_VERSION', 'Not found')}")
print(f"ENDPOINT: {os.getenv('AZURE_OPENAI_ENDPOINT', 'Not found')}")
print(f"DEPLOYMENT: {os.getenv('AZURE_OPENAI_DEPLOYMENT', 'Not found')}")

Looking for .env file at: /home/chicha/repos/openai-sdk-101/01_hello_agent/.env
Does .env file exist? True
API KEY: 2geV8...
API VERSION: 2025-01-01-preview
ENDPOINT: https://open-aj.openai.azure.com
DEPLOYMENT: gpt-4o


In [ ]:
# Configure logging to disable verbose HTTP request logs from OpenAI client
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

try:
    azure_openai_client = AsyncAzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY", ""),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION", ""),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", ""),
        azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT", ""),
    )
    print("Azure OpenAI client created successfully")
except Exception as e:
    print(f"Error creating Azure OpenAI client: {e}")

Azure OpenAI client created successfully


In [ ]:
# Test if the client works with a simple completion
try:
    response = await azure_openai_client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT", ""),
        messages=[{"role": "user", "content": "Say hello"}],
    )
    print("Client test successful:")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Error testing Azure OpenAI client: {e}")

Client test successful:
Hello! How can I assist you today? 😊


In [ ]:
try:
    model = OpenAIChatCompletionsModel(
        model="gpt-4o", openai_client=azure_openai_client
    )
    print("Model created successfully")
    config = RunConfig(
        model=model,
        tracing_disabled=True,
    )
    print("Config created successfully")
except Exception as e:
    print(f"Error creating model or config: {e}")

Model created successfully
Config created successfully


In [26]:
try:
    agent = Agent(
        name="Master Agent",
        model=model,
    )
    print("Agent created successfully")
except Exception as e:
    print(f"Error creating agent: {e}")

Agent created successfully


In [27]:
try:
    # Use the async version instead of run_sync to avoid event loop conflicts
    async def run_agent():
        result = await Runner.run(agent, "Hello, how are you.", run_config=config)
        print("\nCALLING AGENT\n")
        print(result.final_output)
        return result

    # Execute the async function directly - Jupyter will handle it properly
    result = await run_agent()
except Exception as e:
    import traceback

    print(f"Error running agent: {e}")
    print(traceback.format_exc())


CALLING AGENT

Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How are *you* doing today? 😊


In [ ]:
agent = Agent(name="Assistant", instructions="You only respond in haikus.")
result = await Runner.run(agent, "Tell me about yourself.", run_config=config)
print(result.final_output)


Words weave through my mind,  
A guide born to help and share,  
Whispers in the wind.


In [29]:
set_default_openai_client(client=azure_openai_client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

In [30]:
@function_tool
def get_weather(location: str) -> str:
    """
    Get the weather for a given location.
    """
    return f"The weather in {location} is sunny with a high of 75°F."

In [ ]:
agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant.",
    tools=[get_weather],
)
result = await Runner.run(agent, "What is the weather in New York?", run_config=config)
print(result.final_output)

The weather in New York is currently sunny with a high of 75°F.


In [37]:
async def main():
    agent = Agent(name="Assistant", instructions="You only respond in haikus.")
    result = Runner.run_streamed(agent, "Tell me about recursion in programming.", run_config=config)
    #print(result.final_output)
    async for message in result.stream_events():
        print(message, end="", flush=True)

await main()

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions='You only respond in haikus.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')

RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='__fake_id__', created_at=1745076683.3897088, error=None, incomplete_details=None, instructions=None, metadata=None, model='gpt-4o', object='response', output=[], parallel_tool_calls=False, temperature=None, tool_choice='auto', tools=[], top_p=None, max_output_tokens=None, previous_response_id=None, reasoning=None, service_tier=None, status=None, text=None, truncation=None, usage=None, user=None), type='response.created'), type='raw_response_event')RawResponsesStreamEvent(data=ResponseOutputItemAddedEvent(item=ResponseOutputMessage(id='__fake_id__', content=[], role='assistant', status='in_progress', type='message'), output_index=0, type='response.output_item.added'), type='raw_response_event')RawResponsesStreamEvent(data=ResponseContentPartAddedEvent(content_index=0, item_id='__fake_id__', output_index=0, part=ResponseOutputText(annotations=[], text='', type='output_text'), type='response.content_part.added'), type